In [2]:
import pandas as pd
import os

dyad_df = pd.read_parquet('data_dyad_monthly/dyad_df.parquet')
# get only dyads from Jan 2016 to Jan 2019
# to date
dyad_df['date'] = pd.to_datetime(dyad_df['date'])
dyad_df = dyad_df[
    (dyad_df['date'] >= pd.Timestamp(year=2016, month=7, day=1)) &
    (dyad_df['date'] <= pd.Timestamp(year=2019, month=1, day=1))]
dyad_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 562495 entries, 5526414 to 6088908
Columns: 263 entries, month_id to ratio_wdi_ms_mil_xpnd_zs
dtypes: datetime64[ns](1), float64(255), int64(5), object(2)
memory usage: 1.1+ GB


In [3]:
dyad_df

,month_id,date,country_id_a,country_id_b,a_country_name,b_country_name,a_gleditsch_ward,a_ged_sb,a_ged_ns,a_ged_os,...,b_renewable_internal_pcap_t48,b_renewable_pcap_t48,b_splag_1_decay_ged_sb_5,b_splag_1_decay_ged_os_5,b_splag_1_decay_ged_ns_5,ratio_wdi_sp_pop_totl,ratio_ged_sb,ratio_wdi_sp_dyn_imrt_in,ratio_vdem_v2x_ex_military,ratio_wdi_ms_mil_xpnd_zs
5526414,439,2016-07-01,1,2,Guyana,Suriname,110,0.0,0.0,0.0,...,182911.620431,182911.620431,0.000006,0.128716,0.749157,1.312014,0.000000,1.497268,0.000000,4.957652
5526415,439,2016-07-01,1,3,Guyana,Trinidad and Tobago,110,0.0,0.0,0.0,...,2855.402846,2855.402846,0.000000,0.000000,0.000000,0.517082,0.000000,1.565714,0.000000,2.305654
5526416,439,2016-07-01,1,4,Guyana,Venezuela,110,0.0,0.0,0.0,...,27417.474509,45128.141272,0.971538,0.614482,0.776997,0.024731,0.000000,1.660606,0.000000,4.957652
5526417,439,2016-07-01,1,5,Guyana,Samoa,110,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,3.708932,0.000000,1.723270,0.000000,4.957652
5526418,439,2016-07-01,1,6,Guyana,Tonga,110,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,7.114745,0.000000,2.584906,0.000000,4.957652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6088904,469,2019-01-01,243,245,Morocco,Sudan,600,0.0,0.0,0.0,...,102.819966,971.648680,6.149093,5.183314,4.848733,0.855436,0.000000,0.416667,0.497006,0.992970
6088905,469,2019-01-01,243,246,Morocco,South Sudan,600,0.0,0.0,0.0,...,2426.355899,4619.408346,4.719313,5.131790,5.177793,3.456121,0.000000,0.274295,0.415000,1.067648
6088906,469,2019-01-01,244,245,Mauritania,Sudan,435,0.0,0.0,0.0,...,102.819966,971.648680,6.149093,5.183314,4.848733,0.101686,0.000000,0.826190,2.497006,1.062418
6088907,469,2019-01-01,244,246,Mauritania,South Sudan,435,0.0,0.0,0.0,...,2426.355899,4619.408346,4.719313,5.131790,5.177793,0.410830,0.000000,0.543887,2.085000,1.142318


In [ ]:
from utilities import views_month_id_to_date

# load benchmark model
prediction_year = 2019
model_names = {
    "bootstrap": "bm_cm_bootstrap_expanded_",
    "poisson": "bm_cm_last_historical_poisson_expanded_",
}
benchmark_model = pd.read_parquet(f'Benchmarks/{model_names["bootstrap"]}{prediction_year}.parquet')
# there is 20 draws per each country per each month. Get the mean of the draws
benchmark_model = benchmark_model.groupby(['month_id', 'country_id']).mean().reset_index()
# add date column
benchmark_model['date'] = views_month_id_to_date(benchmark_model['month_id'])
benchmark_model

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
# countries_encoded = encoder.fit_transform(dyad_df[['country_id_a', 'country_id_b']])
# countries_encoded = encoder.transform(dyad_df[['country_id_a', 'country_id_b']])
# countries_encoded = pd.DataFrame(countries_encoded, columns=encoder.get_feature_names(['country_id_a', 'country_id_b']))
# countries_encoded
# trasnform country_id_a and country_id_b to one hot encoding and remove the original columns
# country_a_and_b_ids = dyad_df[dyad_df['country_id_a'], dyad_df['country_id_b']]


# country_a_and_b_ids = dyad_df[['country_id_a', 'country_id_b']]
# dyad_df = pd.get_dummies(dyad_df, columns=['country_id_a', 'country_id_b'], drop_first=False, dtype=int)
# # merge back country_id_a and country_id_b
# dyad_df = pd.concat([dyad_df, country_a_and_b_ids], axis=1)
# dyad_df

In [7]:
cut_year = prediction_year - 2  # 2016

features_to_oct = pd.Timestamp(year=cut_year, month=10, day=1)  # 2016-Oct-01

# Splitting the dataset
train_df = dyad_df[dyad_df['date'] <= features_to_oct]
# test_df is one year from Oct 2016 to Oct 2017
test_df = dyad_df[
    (dyad_df['date'] >= pd.Timestamp(year=prediction_year - 2, month=10, day=1)) &  # oct 2016 predicts Jan 2018
    (dyad_df['date'] < pd.Timestamp(year=prediction_year - 1, month=10, day=1))]  # oct 2017 predicts Jan 2019


In [ ]:
# Explicitly list columns to be dropped before scaling
columns_to_drop = ['country_id_a', 'country_id_b', 'date', 'a_ged_sb', 'b_ged_sb', 'a_gleditsch_ward',
                   'b_gleditsch_ward', 'a_country_name', 'b_country_name']
columns_to_re_add = ['a_ged_sb', 'b_ged_sb']

# Also, drop one-hot encoded country identifiers if they are already in the dataframe
columns_to_re_add.extend(dyad_df.filter(regex='^country_id_a_').columns.tolist())
columns_to_re_add.extend(dyad_df.filter(regex='^country_id_b_').columns.tolist())

# Determine numeric columns by excluding the ones to drop from the dataframe
numeric_columns = dyad_df.drop(columns=columns_to_drop).drop(columns_to_re_add).columns.tolist()

In [ ]:
from sklearn.preprocessing import StandardScaler

TARGET_COLUMNS = ['a_ged_sb', 'b_ged_sb']

# Initialize the scaler using only the training data
scaler = StandardScaler().fit(train_df.drop(TARGET_COLUMNS, axis=1))

# Apply the transformations to both the training and test sets
train_df_scaled = scaler.transform(train_df.drop(TARGET_COLUMNS, axis=1))
test_df_scaled = scaler.transform(test_df.drop(TARGET_COLUMNS, axis=1))

# Reconstruct the dataframes with normalized features and the targets
train_df = pd.DataFrame(train_df_scaled, columns=train_df.columns.drop(TARGET_COLUMNS))
train_df[TARGET_COLUMNS] = train_df[TARGET_COLUMNS]

test_df = pd.DataFrame(test_df_scaled, columns=test_df.columns.drop(TARGET_COLUMNS))
test_df[TARGET_COLUMNS] = test_df[TARGET_COLUMNS]

In [ ]:
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader

torch.manual_seed(42)


class CountryDyadDataset(Dataset):
    def __init__(self, dataframe, feature_columns, target_columns=None):
        if target_columns is None:
            target_columns = TARGET_COLUMNS
        self.features = torch.tensor(dataframe[feature_columns].values, dtype=torch.float)
        self.targets = torch.tensor(dataframe[target_columns].values, dtype=torch.float)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

In [ ]:
# Update feature_columns to exclude only the targets and other non-feature columns, based on the final dataframe
feature_columns = [col for col in train_df_final.columns if col not in ['a_ged_sb', 'b_ged_sb']]

train_dataset = CountryDyadDataset(train_df_final, feature_columns)
test_dataset = CountryDyadDataset(test_df_final, feature_columns)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64)

In [ ]:


class CountryDyadDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe
        self.features = torch.tensor(self.dataframe.drop(['a_ged_sb', 'b_ged_sb'], axis=1).values, dtype=torch.float)
        self.targets = torch.tensor(self.dataframe[['a_ged_sb', 'b_ged_sb']].values, dtype=torch.float)

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]


# Creating dataset object
data_set = Data()


# Creating a custom Multiple Linear Regression Model
class MultipleLinearRegression(torch.nn.Module):
    # Constructor
    def __init__(self, input_dim, output_dim):
        super(MultipleLinearRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)

    # Prediction
    def forward(self, x):
        y_pred = self.linear(x)
        return y_pred


# Creating the model object
MLR_model = MultipleLinearRegression(2, 2)
print("The parameters: ", list(MLR_model.parameters()))

# defining the model optimizer
optimizer = torch.optim.SGD(MLR_model.parameters(), lr=0.1)
# defining the loss criterion
criterion = torch.nn.MSELoss()

# Creating the dataloader
train_loader = DataLoader(dataset=data_set, batch_size=64)

# Train the model
losses = []
epochs = 20
for epoch in range(epochs):
    for x, y in train_loader:
        y_pred = MLR_model(x)
        loss = criterion(y_pred, y)
        losses.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"epoch = {epoch}, loss = {loss}")
print("Done training!")

# Plot the losses
plt.plot(losses)
plt.xlabel("no. of iterations")
plt.ylabel("total loss")
plt.show()